## MSc Data Analytics - Capstone Project 

## Predictive Insights in the Coffee Market: Time Series Models for forecasting of coffee in the commodity stock market

Student id: 2020274 Clarissa Cardoso



This notebook will serve as an experiementation space for using different timeseries models to forecast the prices of coffee in the commodity stock market. The project will use the ICO Composite Indicator Price (I-CIP) of coffee beans available for free in ICO's website under the Public Market Information feature. The ICO Composite Indicator Price serves as a crucial benchmark for understanding the overall trends and movements in the global coffee market, making accurate forecasting essential for stakeholders across the coffee industry.


#### Objectives:

- Building and evaluating time series forecasting models to predict future I-CIP values.
- Assessing the effectiveness and accuracy of different modeling techniques in capturing the dynamics of the coffee market.

#### Methodology:


- Data Collection: gather historical I-CIP data from reliable sources, (spanning multiple years to capture a comprehensive view of market trends. but at least 1 year of data) 
-Data Preprocessing: clean and preprocess the data, handling missing values, outliers, and ensuring consistency for modeling purposes.
- Exploratory Data Analysis (EDA): conduct exploratory analysis to gain insights into the underlying patterns, seasonality, and trends present in the I-CIP data.
- Model Development: build and train various time series forecasting models, including traditional statistical models (e.g., ARIMA) and machine learning algorithms (e.g., LSTM neural networks).
- Model Evaluation: We will evaluate the performance of each model using appropriate metrics, such as mean absolute error (MAE) and root mean squared error (RMSE), to determine their predictive accuracy.
- Forecasting: We will generate forecasts for future I-CIP values using the best-performing model(s) and visualize the results to facilitate interpretation and decision-making.


<br>
<br>
<br><br>
<br>
<br>
<br>
<br>
<br>
<br>






       
         considering the I-CIP values are a weighted average of 4 main types of beans commercialised(Colombian Milds,  Other Milds,  Brazilian Naturals, Robustas), how would the diffenret weights present during the eda, and how they could affect the final icip values?



#### Importing required libraries for the project

In [1]:
#importing libraries
import warnings
warnings.filterwarnings("ignore")

import pandas as pd #dataframes 
import numpy as np #linear algebra
import seaborn as sns #visualization
sns.set(color_codes=True)

import scipy.stats as stats #statistical resources

import matplotlib.pyplot as plt #visualisation 
%matplotlib inline 
import matplotlib.pyplot as plt

from matplotlib import colors
from matplotlib.ticker import PercentFormatter


In [2]:
import os
# Listing all the files in the folder
os.listdir("icip_2023") 

## folder contains the free data available in ICO's website, contains data from Feb/23 to Feb/24

['I-CIP_August_2023.csv',
 'I-CIP_September_2023.csv',
 '.DS_Store',
 'I-CIP_April_2023.csv',
 'I-CIP_March_2023.csv',
 'I-CIP_November_2023.csv',
 'I-CIP_Febuary_2024.csv',
 'I-CIP_January_2024.csv',
 'I-CIP_Febuary_2023.csv',
 'I-CIP_October_2023.csv',
 'I-CIP_June_2023.csv',
 'I-CIP_July_2023.csv',
 'I-CIP_December_2023.csv',
 'I-CIP_May_2023.csv']

Files are not in cronological orders, will affect the loop for calling the separate csv's, and cant import them  at the same time to concat them in one single dataframe?

this means the position of each file in the list is as: 

- 'I-CIP_August_2023.csv', [0]
- 'I-CIP_September_2023.csv',[1]
- '.DS_Store',
- 'I-CIP_April_2023.csv',[2]
- 'I-CIP_March_2023.csv',[3]
- 'I-CIP_November_2023.csv',[4]
- 'I-CIP_Febuary_2024.csv',[5] 
 

....
 


In [3]:
#create for loop to import csv files from the folder with less comands.

# create an empty list to store dfs
dataframes = []

# path to folder where csv files are (in this case same directory)
folder_path = "icip_2023"

for file in os.listdir(folder_path):
    if file.endswith(".csv"): #consider only csv files, ignore .DS_Store
        file_path = os.path.join(folder_path, file) #  Construct the full file path
        dataframes.append(pd.read_csv(file_path)) # Read the CSV file and append it to the list

In [4]:
#check the lenght of the directory, how many files exist in the folder
len(dataframes)

13

In [5]:
#check if order of files correspond with the directory list, testing if loop is working
dataframes[9].head()

Unnamed: 0 Unnamed: 1 Colombian   Unnamed: 3 Brazilian Unnamed: 5
0        NaN      I-CIP       NaN  Other Milds       NaN   Robustas
1        NaN        NaN     Milds          NaN  Naturals        NaN
2     01-Jun     173.56    220.40       215.27    181.78     126.54
3     02-Jun     175.62    222.41       216.73    181.98     130.57
4     05-Jun     173.15    219.54       213.37    180.94     127.53

In [6]:
#define a variable name for each dataset to facilitate data manipulation

feb24= dataframes[5]

In [7]:
#cheack first 5 rows of last dataset in the file corresponds to the most recent values from feb24

feb24.info()
print(feb24.shape)
print(feb24.isnull().sum())
feb24.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  21 non-null     object
 1   Unnamed: 1  22 non-null     object
 2   Colombian   22 non-null     object
 3   Unnamed: 3  22 non-null     object
 4   Brazilian   22 non-null     object
 5   Unnamed: 5  22 non-null     object
dtypes: object(6)
memory usage: 1.2+ KB
(23, 6)
Unnamed: 0    2
Unnamed: 1    1
Colombian     1
Unnamed: 3    1
Brazilian     1
Unnamed: 5    1
dtype: int64


Unnamed: 0 Unnamed: 1 Colombian   Unnamed: 3 Brazilian Unnamed: 5
0        NaN      I-CIP       NaN  Other Milds       NaN   Robustas
1        NaN        NaN     Milds          NaN  Naturals        NaN
2     01-Feb     185.24    213.15       212.01    190.15     156.09
3     02-Feb     183.69    212.32       212.94    188.41     153.01
4     05-Feb     180.93    210.11       209.12    185.69     150.65

importing a random dataset to see how the columsn are presented: a messy header and real data starts fromm third row.
First thing to adjust: collumn names and remove NaN values from second row.
- from the basic info function we can see the dates are not in correct format (need to convert to DD-MM-AAAA instead of 01-Feb) and all the values are not in int/float format, they are objects.

I want to combine bthe data from all csv files into a single dataframe for data manipulation but when importing them as panda df, need to make other adjustments - glob?? (https://stackabuse.com/bytes/importing-multiple-csv-files-into-a-single-dataframe-using-pandas-in-python/)

Common Errors as stated by Saturn Cloud (https://saturncloud.io/blog/loading-multiple-csv-files-from-a-folder-into-one-dataframe-a-comprehensive-guide/#1) 

    Incorrect file path: Not setting the working directory correctly or providing an invalid path to the folder containing the CSV files.

    Column mismatch: Assuming all files have the same column names and order. This can lead to errors when merging DataFrames with different structures.




In [8]:

        
### # Concatenate all DataFrames into a single DataFrame and remove null values
###merged_df = pd.concat(dataframes, ignore_index=True, dropna=True) could not include dropna in the same line because 
## concat function doesnt alow this method


# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(dataframes, ignore_index=True)

# Remove null values from the concatenated DataFrame
#merged_df.dropna(inplace=True) this function removed all values, leaving an emppty df. 

# Display the merged DataFrame
print(merged_df)

    Unnamed: 0 Unnamed: 1 Colombian   Unnamed: 3 Brazilian Unnamed: 5  \
0          NaN      I-CIP       NaN  Other Milds       NaN   Robustas   
1          NaN        NaN     Milds          NaN  Naturals        NaN   
2       01-Aug     161.70    196.62       197.67    164.03     127.08   
3       02-Aug     163.62    198.66       199.72    166.00     128.82   
4       03-Aug     161.68    196.28       197.32    163.52     127.77   
..         ...        ...       ...          ...       ...        ...   
300     25-May        NaN    222.17       217.37    182.38     123.94   
301     26-May        NaN    220.39       215.81    180.94     123.98   
302     29-May        NaN    220.52       215.05    181.73     124.68   
303     30-May        NaN    215.91       210.53    177.23     124.47   
304     31-May        NaN    216.84       211.48    178.19     124.45   

    ICO Composite  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4             NaN  
.. 

setting dropna=True, any rows with null values will be removed from the concatenated DataFrame. However this function caused an empty dataframe? Also, the files are still not in cronological order, so need to find a way to import them in the correct order. When we remove the dropna function from the merged_df, at the same time the values get misplaced into a new column, which is also not adequate for manipulation.

In [9]:
merged_df.head()

Unnamed: 0 Unnamed: 1 Colombian   Unnamed: 3 Brazilian Unnamed: 5  \
0        NaN      I-CIP       NaN  Other Milds       NaN   Robustas   
1        NaN        NaN     Milds          NaN  Naturals        NaN   
2     01-Aug     161.70    196.62       197.67    164.03     127.08   
3     02-Aug     163.62    198.66       199.72    166.00     128.82   
4     03-Aug     161.68    196.28       197.32    163.52     127.77   

  ICO Composite  
0           NaN  
1           NaN  
2           NaN  
3           NaN  
4           NaN

In [10]:
merged_df.tail()

Unnamed: 0 Unnamed: 1 Colombian Unnamed: 3 Brazilian Unnamed: 5  \
300     25-May        NaN    222.17     217.37    182.38     123.94   
301     26-May        NaN    220.39     215.81    180.94     123.98   
302     29-May        NaN    220.52     215.05    181.73     124.68   
303     30-May        NaN    215.91     210.53    177.23     124.47   
304     31-May        NaN    216.84     211.48    178.19     124.45   

    ICO Composite  
300        173.46  
301        172.49  
302        172.84  
303        169.87  
304        170.47

Before continuing, ill try to first rename the columns and remove the NaN values in each dataframe separately, to ensure all dfs will be in the same shape to facilitate concatenation, also might be easier to import them in the correct order once this is done?


In [11]:
# Define a list of new column names
new_column_names = {'Unnamed: 0': 'date', 
                    'Unnamed: 1': 'I-CIP', 'Colombian':'colombian_milds', 
                    'Unnamed: 3':'other_milds', 'Brazilian':'brazilian_nat', 'Unnamed: 5':'robustas'}


# Apply changes to each DataFrame separately
for df in dataframes:
    # Rename columns
    df.rename(columns=new_column_names, inplace=True)
    
    # Remove NaN values
  #  df.dropna(inplace=True) this one makes all dataframes empty again.

In [12]:
print(len(dataframes)) #check is list is not empty anymore

13


In [13]:
## can see the changes were applied and that also all dfs follow the same pattern of having NaN values on the 
## first 2 rows, so i can either perfom a similar loop to drop these rows and merge them in the correct order?
print(dataframes[1].head())
print(dataframes[2].head())
print(dataframes[4].head())
print(dataframes[8].head())
print(dataframes[6].head())
print(dataframes[3].head())
print(dataframes[10].head())

     date   I-CIP colombian_milds  other_milds brazilian_nat  robustas
0     NaN   I-CIP             NaN  Other Milds           NaN  Robustas
1     NaN     NaN           Milds          NaN      Naturals       NaN
2  01-Sep  152.36          184.05       183.14        151.88    124.26
3  04-Sep  152.01          183.75       182.60        151.43    124.09
4  05-Sep  153.02          185.15       184.01        152.88    124.34
     date      ICO Composite colombian_milds  other_milds brazilian_nat  \
0     NaN                NaN             NaN  Other Milds           NaN   
1     NaN  Indicator (I-CIP)           Milds          NaN      Naturals   
2  03-Apr             168.87          222.30       219.78        183.11   
3  04-Apr             168.14          220.82       218.29        181.52   
4  05-Apr             172.44          225.91       223.38        186.70   

   robustas  
0  Robustas  
1       NaN  
2    109.10  
3    109.80  
4    112.63  
     date   I-CIP colombian_milds  othe

In [14]:
# Drop the first two rows in each DataFrame
for df in dataframes:
    df.drop(df.index[:2], inplace=True)

In [15]:
#print test across different months to see changes
print(dataframes[1].head())
print(dataframes[2].head())
print(dataframes[4].head())
print(dataframes[8].head())

     date   I-CIP colombian_milds other_milds brazilian_nat robustas
2  01-Sep  152.36          184.05      183.14        151.88   124.26
3  04-Sep  152.01          183.75      182.60        151.43   124.09
4  05-Sep  153.02          185.15      184.01        152.88   124.34
5  06-Sep  153.39          186.34      185.01        154.83   122.74
6  07-Sep  151.58          181.98      185.21        151.52   121.88
     date ICO Composite colombian_milds other_milds brazilian_nat robustas
2  03-Apr        168.87          222.30      219.78        183.11   109.10
3  04-Apr        168.14          220.82      218.29        181.52   109.80
4  05-Apr        172.44          225.91      223.38        186.70   112.63
5  06-Apr        173.75          228.43      225.90        189.30   111.71
6  07-Apr        173.96          228.72      226.04        189.39   112.03
     date   I-CIP colombian_milds other_milds brazilian_nat robustas
2  01-Nov  153.32          187.15      186.58        158.95   117.8

In [16]:
#define a variable name for each dataset to facilitate data manipulation
# but now for each month, give them a variable so i can concat them in the correct order
# might be a but extra work but the lambda function didnt work even after renaming the original files and adding '_'
# also the natsorted library was not enabled in my machine
feb24= dataframes[5]
jan24=dataframes[6]
dec23=dataframes[11]
nov23=dataframes[4]
oct23=dataframes[8]
sep23=dataframes[1]
aug23=dataframes[0]
jul23=dataframes[10]
jun23=dataframes[9]
may23=dataframes[12]
apr23=dataframes[2]
mar23=dataframes[3]
feb23=dataframes[7]

In [17]:
print(dataframes)



[      date   I-CIP colombian_milds other_milds brazilian_nat robustas
2   01-Aug  161.70          196.62      197.67        164.03   127.08
3   02-Aug  163.62          198.66      199.72        166.00   128.82
4   03-Aug  161.68          196.28      197.32        163.52   127.77
5   04-Aug  158.15          192.33      192.15        159.20   126.01
6   07-Aug  160.76          194.43      194.65        161.75   128.91
7   08-Aug  160.03          192.70      193.63        160.61   129.03
8   09-Aug  158.62          191.14      191.37        158.39   128.88
9   10-Aug  158.26          191.29      191.28        158.61   127.69
10  11-Aug  155.96          188.35      187.47        155.31   127.33
11  14-Aug  151.31          183.40      182.46        149.96   123.60
12  15-Aug  150.92          182.53      181.59        149.34   123.86
13  16-Aug  149.07          181.45      180.49        147.98   120.90
14  17-Aug  149.34          181.10      179.23        148.87   121.72
15  18-Aug  149.68 

#another issue was identified from the original data here
the first 4 months contain a different column for the icip prices, and the mergeddf added a column with  missing values in place for the 'ghost' feature, it seems to have 2 diffent names even after i renamed the collumns in previuos lines (ICO Composite and I-CIP). So before i merge them toghether in cronolgical order, ill adjust these features. Or could i just merge the dfs without the labels? 

In [18]:
# Define the list of DataFrames in the desired order
dfs_in_order = [feb23, mar23, apr23, may23, jun23, jul23, aug23, sep23, oct23, nov23, dec23, jan24, feb24]

# Concatenate the DataFrames
merged_df = pd.concat(dfs_in_order,ignore_index=True)

# Display the merged DataFrame
merged_df

date ICO Composite colombian_milds other_milds brazilian_nat robustas  \
0    01-Feb        171.43          235.92      223.22        191.65   102.31   
1    02-Feb        172.50          237.34      226.26        192.86   102.00   
2    03-Feb        169.47          232.24      221.86        188.61   101.52   
3    06-Feb        171.29          235.17      224.80        190.77   102.02   
4    07-Feb        172.14          235.65      226.72        191.92   102.10   
..      ...           ...             ...         ...           ...      ...   
274  23-Feb           NaN          204.48      202.46        183.74   149.40   
275  26-Feb           NaN          204.12      202.11        183.48   149.08   
276  27-Feb           NaN          207.11      205.10        186.34   151.51   
277  28-Feb           NaN          206.01      204.00        185.39   150.23   
278  29-Feb           NaN          207.67      205.52        186.73   150.40   

      I-CIP  
0       NaN  
1       NaN  
2       NaN  
3       NaN  
4       NaN  
..      ...  
274  177.80  
275  177.48  
276  180.23  
277  179.12  
278  180.11  

[279 rows x 7 columns]

In [19]:
# Rename columns of the first four DataFrames to match the later DataFrames
feb23.rename(columns={'Unnamed: 0': 'date', 'ICO Composite': 'I-CIP'}, inplace=True)
mar23.rename(columns={'Unnamed: 0': 'date', 'ICO Composite': 'I-CIP'}, inplace=True)
apr23.rename(columns={'Unnamed: 0': 'date', 'ICO Composite': 'I-CIP'}, inplace=True)
may23.rename(columns={'Unnamed: 0': 'date', 'ICO Composite': 'I-CIP'}, inplace=True)

# Concatenate the DataFrames in the desired order
merged_df = pd.concat([feb23, mar23, apr23, may23, jun23, jul23, aug23, sep23, oct23, nov23, dec23, jan24, feb24], ignore_index=True)

# Display the merged DataFrame
print(merged_df)

       date   I-CIP colombian_milds other_milds brazilian_nat robustas
0    01-Feb  171.43          235.92      223.22        191.65   102.31
1    02-Feb  172.50          237.34      226.26        192.86   102.00
2    03-Feb  169.47          232.24      221.86        188.61   101.52
3    06-Feb  171.29          235.17      224.80        190.77   102.02
4    07-Feb  172.14          235.65      226.72        191.92   102.10
..      ...     ...             ...         ...           ...      ...
274  23-Feb  177.80          204.48      202.46        183.74   149.40
275  26-Feb  177.48          204.12      202.11        183.48   149.08
276  27-Feb  180.23          207.11      205.10        186.34   151.51
277  28-Feb  179.12          206.01      204.00        185.39   150.23
278  29-Feb  180.11          207.67      205.52        186.73   150.40

[279 rows x 6 columns]


## EDA

Perfom some basic eda to see how the data is presented after the merge
However, when looking at the .info output, before obtaining any sort of relevant eda i need to make the correct conversions for datatypes that can be used foer forecasting. It was also noted that having the dates as 01-Feb is not at all helpfull, so first need to add the year to facilitate the conversion https://pandas.pydata.org/docs/user_guide/timeseries.html It also does not consider the year of each datapoint. 



In [20]:
# Summary Statistics
print(merged_df.info())
merged_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             279 non-null    object
 1   I-CIP            279 non-null    object
 2   colombian_milds  279 non-null    object
 3   other_milds      279 non-null    object
 4   brazilian_nat    279 non-null    object
 5   robustas         279 non-null    object
dtypes: object(6)
memory usage: 13.2+ KB
None


date   I-CIP colombian_milds other_milds brazilian_nat robustas
count      279     279             279         279           279      279
unique     263     270             274         273           272      267
top     01-Feb  173.84          192.70      212.69        180.94   126.65
freq         2       2               2           3             2        3

In [21]:
# Convert numerical columns to numeric type

#merged_df['date'] = pd.to_datetime(merged_df['date']) 

merged_df['I-CIP'] = pd.to_numeric(merged_df['I-CIP'])
merged_df['colombian_milds'] = pd.to_numeric(merged_df['colombian_milds'])
merged_df['other_milds'] = pd.to_numeric(merged_df['other_milds'])
merged_df['brazilian_nat'] = pd.to_numeric(merged_df['brazilian_nat'])
merged_df['robustas'] = pd.to_numeric(merged_df['robustas'])

# Confirm the data types after conversion
print(merged_df.dtypes)

date                object
I-CIP              float64
colombian_milds    float64
other_milds        float64
brazilian_nat      float64
robustas           float64
dtype: object


In [24]:
merged_df.head(30)

date   I-CIP  colombian_milds  other_milds  brazilian_nat  robustas
0   01-Feb-2023  171.43           235.92       223.22         191.65    102.31
1   02-Feb-2023  172.50           237.34       226.26         192.86    102.00
2   03-Feb-2023  169.47           232.24       221.86         188.61    101.52
3   06-Feb-2023  171.29           235.17       224.80         190.77    102.02
4   07-Feb-2023  172.14           235.65       226.72         191.92    102.10
5   08-Feb-2023  171.23           234.06       225.13         190.38    102.35
6   09-Feb-2023  169.89           232.17       223.54         188.61    101.71
7   10-Feb-2023  170.31           233.38       223.14         189.73    101.76
8   13-Feb-2023  171.50           235.34       224.91         191.24    102.06
9   14-Feb-2023  175.86           241.56       231.13         197.28    103.26
10  15-Feb-2023  171.73           235.53       225.11         191.36    102.43
11  16-Feb-2023  173.91           238.50       228.08         194.25    103.26
12  17-Feb-2023  177.59           242.86       233.37         199.53    104.40
13  20-Feb-2023  177.67           241.26       235.07         198.15    105.35
14  21-Feb-2023  180.49           245.25       239.04         202.09    106.15
15  22-Feb-2023  183.85           248.76       242.56         205.62    109.23
16  23-Feb-2023  180.91           245.20       238.99         202.07    107.37
17  24-Feb-2023  178.96           242.39       234.37         200.55    106.89
18  27-Feb-2023  177.25           237.56       233.63         198.41    106.04
19  28-Feb-2023  177.42           237.60       233.67         198.53    106.38
20  01-Mar-2023  175.93           234.63       230.69         195.40    107.65
21  02-Mar-2023  175.08           232.62       229.73         193.63    108.05
22  03-Mar-2023  172.26           228.55       225.58         189.86    107.25
23  06-Mar-2023  173.04           228.40       229.01         190.63    106.78
24  07-Mar-2023  174.83           232.32       228.06         195.15    107.13
25  08-Mar-2023  171.66           227.12       222.86         189.90    107.59
26  09-Mar-2023  170.11           224.55       220.75         187.12    107.78
27  10-Mar-2023  171.96           227.72       223.46         190.66    107.24
28  13-Mar-2023  171.67           228.93       224.33         190.82    105.38
29  14-Mar-2023  167.97           223.86       220.89         185.08    103.73

In [25]:
merged_df.tail(30)

date   I-CIP  colombian_milds  other_milds  brazilian_nat  \
249  19-Jan-2024  176.29           204.72       202.60         175.74   
250  22-Jan-2024  179.20           209.12       206.98         179.28   
251  23-Jan-2024  180.39           209.52       207.39         180.52   
252  24-Jan-2024  180.51           208.94       206.81         179.94   
253  25-Jan-2024  181.17           205.30       203.66         182.59   
254  26-Jan-2024  183.39           210.83       209.93         188.33   
255  29-Jan-2024  181.43           207.61       206.71         184.99   
256  30-Jan-2024  184.94           212.12       210.83         188.99   
257  31-Jan-2024  185.54           213.00       211.71         190.01   
258  01-Feb-2023  185.24           213.15       212.01         190.15   
259  02-Feb-2023  183.69           212.32       212.94         188.41   
260  05-Feb-2023  180.93           210.11       209.12         185.69   
261  06-Feb-2023  180.65           208.72       207.73         184.41   
262  07-Feb-2023  182.67           210.75       210.53         187.07   
263  08-Feb-2023  181.62           209.01       208.65         185.68   
264  09-Feb-2023  187.07           214.58       215.14         191.21   
265  12-Feb-2023  186.76           214.42       214.67         191.90   
266  13-Feb-2023  184.47           211.53       211.79         188.97   
267  14-Feb-2023  179.87           206.31       206.57         183.73   
268  15-Feb-2023  181.84           208.63       208.90         186.19   
269  16-Feb-2023  183.01           210.82       210.13         187.06   
270  19-Feb-2023  183.39           210.72       209.88         186.88   
271  20-Feb-2023  182.73           210.52       209.69         186.90   
272  21-Feb-2023  184.10           211.97       211.14         188.16   
273  22-Feb-2023  180.02           207.20       206.37         183.38   
274  23-Feb-2023  177.80           204.48       202.46         183.74   
275  26-Feb-2023  177.48           204.12       202.11         183.48   
276  27-Feb-2023  180.23           207.11       205.10         186.34   
277  28-Feb-2023  179.12           206.01       204.00         185.39   
278  29-Feb-2023  180.11           207.67       205.52         186.73   

     robustas  
249    151.94  
250    152.96  
251    154.82  
252    156.19  
253    158.78  
254    154.50  
255    154.88  
256    157.30  
257    157.28  
258    156.09  
259    153.01  
260    150.65  
261    152.25  
262    153.27  
263    153.22  
264    157.98  
265    156.83  
266    155.65  
267    152.16  
268    153.38  
269    154.43  
270    155.84  
271    154.16  
272    155.55  
273    152.69  
274    149.40  
275    149.08  
276    151.51  
277    150.23  
278    150.40